# proccess json out

In [1]:
import json
import os
from mmlu_to_bedrock import get_mmlu
from collections import defaultdict
from tqdm import tqdm
import re

/opt/anaconda3/envs/prm_train/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fail_count = 0

def extract_answer(text):
    pattern = r"answer is \(?([A-J])\)?"
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        # print("1st answer extract failed\n" + text)
        
    

        return extract_again(text)


def extract_again(text):
    match = re.search(r'.*[aA]nswer:\s*([A-J])', text)
    if match:
        return match.group(1)
    else:
        return None
        return extract_final(text)


def extract_final(text):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(0)
    else:
        return None

In [3]:
def get_data(data_dir):
    '''
    aggregates all jsonl file in directory into one list
    '''
    data = []
    for filename in os.listdir(data_dir):

        if filename.endswith('.jsonl.out'):

            file_path = f'{data_dir}/{filename}'
            with open(file_path, 'r') as f:
                for l in f:
                    data.append(json.loads(l))

    return data

def dataset_to_id_dict(ds):

    id_dict = {}

    for d in ds:
        
        # add in chain of thoughts key
        d['chain_of_thoughts'] = []
        id_dict[d['id']] = d

    return id_dict

def get_datasets_by_id(samples=10):



    ds = get_mmlu(samples=samples)


    id_dict = dataset_to_id_dict(ds)


    return id_dict

def parse_bedrock_id(recordId):

    id, cot_id = recordId.split('+')

    return id, cot_id

def parse_cot(cot_str, delimiter='\n\n'):
    '''
        splits cot string into list
        parses out the generated answer
    '''

    cot_splitted = cot_str.strip().split(delimiter)
    cot_splitted = [item.strip() for item in  cot_splitted]
    parsed_answer = extract_answer(cot_splitted[-1])
    return cot_splitted, parsed_answer


def flatten_datasets_by_id(datasets_by_id, dump=True, dump_dir='.'):



        datasets_by_id =list(datasets_by_id.values())

        if dump:
            with open(f'{dump_dir}/cot.json', 'w') as f:
                json.dump(datasets_by_id, f, indent=2)

In [4]:
samples = 150
num_gen = 64


output_directory = f'./bedrock_outputs/mmlu_{samples}_{num_gen}'

bedrock_data = get_data(output_directory)
datasets_by_id = get_datasets_by_id(samples=samples)


100%|██████████| 12032/12032 [00:00<00:00, 1087204.66it/s]


In [7]:

fo = []
failed_len = []
failed_parse = []

for d in tqdm(bedrock_data):

    id, cot_id = parse_bedrock_id(d['recordId'])
    # if 'chain_of_thoughts' not in datasets_by_id[dataset][split][id]:
    #    datasets_by_id[dataset][split][id]['chain_of_thoughts'] = []

    if 'modelOutput' not in d:
        fo.append(d)
        continue

    stop_reason = d['modelOutput']['stop_reason']

    # we want to ignore bad CoT
    if stop_reason == 'length':
        d.pop('modelOutput')
        failed_len.append(d)
        continue

    steps, parsed_answer = parse_cot(d['modelOutput']['generation'])
    if parsed_answer == None: # again to ignore bad CoT
        d.pop('modelOutput')
        failed_parse.append(d)
        continue



    datasets_by_id[id]['chain_of_thoughts'].append({'steps':steps, 
                                        'parsed_answer':parsed_answer,
                                        'parsed_answer_correctness': parsed_answer==datasets_by_id[id]['answer'],
                                        'cot_id':cot_id})
    


cot_output_path = f'cot_data/mmlu_{samples}_{num_gen}/'
os.makedirs(os.path.dirname(cot_output_path), exist_ok=True)
flatten_datasets_by_id(datasets_by_id, dump=True, dump_dir=cot_output_path)

    

100%|██████████| 134400/134400 [00:00<00:00, 150546.74it/s]


In [10]:
from collections import defaultdict
counter = defaultdict(int)
for d in failed_len:
    cat = d['recordId'].split('+')[0].split('_')[-1]
    counter[cat] += 1